In [1]:
# !pip install pandas numpy scikit-learn tqdm  # in case you don't have some libraries required to run the code

In [2]:
import warnings
warnings.filterwarnings('ignore')

from utils import *

import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()

## Загрузка данных

Загрузим данные – только необходимые столбцы для решения задачи.

Проанализировал данные с [pandas-profiling](https://github.com/pandas-profiling/pandas-profiling) (но репорт в репозиторий загружать не стал).


In [3]:
types = {
    'os_id': str,
    'country_id': str,
    'campaign_clicks': int,
    'clicks': int
}
columns = ['date_time', *types.keys()]
df = pd.read_csv(
    '../data/data.csv',
    dtype=types,
    usecols=columns,
    parse_dates=[columns[0]]
)
df = df.set_index(columns[0]).sort_index()
df.head()

,campaign_clicks,os_id,country_id,clicks
date_time,,,,
2021-09-01 00:02:49,0,0,7,0
2021-09-26 00:00:00,1,3,0,0
2021-09-26 00:00:00,2,2,15,0
2021-09-26 00:00:00,2,2,5,0
2021-09-26 00:00:00,1,4,13,0


## Предобработка данных

Подготовим данные к тренировке и тестированию (сразу все для упрощения). Добавим фичей – час, день недели, время дня, и закодируем категориальные фичи.

В реальной жизни (если бы нужно было сохранить модель и переиспользовать ее уже на новых данных) энкодер пришлось бы сделать по другому (здесь используется get_dummies из pandas) и сериализовать вместе с моделью.

In [4]:
data = feature_engineering(df)
data.head()

  0%|          | 0/15821472 [00:00<?, ?it/s]

,campaign_clicks,clicks,hour,weekday,os_id_0,os_id_1,os_id_10,os_id_2,os_id_3,os_id_4,...,country_id_6,country_id_7,country_id_8,country_id_9,daytime_Early Morning,daytime_Eve,daytime_Late Night,daytime_Morning,daytime_Night,daytime_Noon
date_time,,,,,,,,,,,,,,,,,,,,,
2021-09-01 00:02:49,0,0,0,2,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2021-09-26 00:00:00,1,0,0,6,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2021-09-26 00:00:00,2,0,0,6,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2021-09-26 00:00:00,2,0,0,6,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2021-09-26 00:00:00,1,0,0,6,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


## Подбор гиперпараметров

Подберем параметр регуляризации для модели – и валидироваться будем на предпоследнем дне (а последний день нужен для финального теста по условию).

In [5]:
df_train = data.loc[:'2021-09-30']
df_test = data.loc['2021-10-01']

c_score = {c: test_model(df_test, create_model(df_train, c=c)) for c in tqdm(np.logspace(-3, 3, 7))}
print(c_score)

  0%|          | 0/7 [00:00<?, ?it/s]

{0.001: 0.17222138494337919, 0.01: 0.17211409931395602, 0.1: 0.17210640031552743, 1.0: 0.17210543141865484, 10.0: 0.1721063040023169, 100.0: 0.17210980521832803, 1000.0: 0.17211244747729113}


In [6]:
train, test = (
    data.loc[:'2021-10-01'].copy(),
    data.loc['2021-10-02'].copy()
)

## Тренировка и тестирование

Данные готовы, параметры подобраны – теперь тренируем новую модель на всех доступных для тренировки данных и тестируем ее.

In [9]:
model = create_model(train, c=1.0)
test_model(test, model)

0.1481375873231281

И чтобы убедиться, что все это было не зря, сравним результат нашей модели с простым бейзлайном

In [8]:
score(test.clicks, [(1 - train.clicks.mean(), train.clicks.mean())] * len(test))

0.15486198009919763